<a href="https://www.kaggle.com/code/giuliobenedetti/utilities-for-vgg?scriptVersionId=156899765" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
from tensorflow.random import set_seed
from os import environ
from random import seed
import numpy as np

# Prepare a reproducible environment for model training and validation
def reproduce_environment(random_state=123):

    # 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
    environ["PYTHONHASHSEED"] = str(random_state)

    # 2. Set the `python` built-in pseudo-random generator at a fixed value
    seed(random_state)

    # 3. Set the `numpy` pseudo-random generator at a fixed value
    np.random.seed(random_state)

    # 4. Set the `tensorflow` pseudo-random generator at a fixed value
    set_seed(random_state)

In [2]:
import tensorflow as tf
import os

# Define sample preprocessing pipeline
def process_path(file_path, class_names, cut=224, S=256, max_delta=0.2):
    
    # Convert the compressed string to a 3D uint8 tensor
    image = tf.io.read_file(file_path)
    image = tf.io.decode_jpeg(image, channels=3)
   
    # Find rescaling factor
    min_side = tf.math.minimum(tf.shape(image)[0], tf.shape(image)[1])
    scale = S / min_side
    
    # Zero-mean the RGB values by ImageNet Mean
    image = tf.keras.applications.vgg16.preprocess_input(image)

    # Compute new dimensions
    new_height = tf.cast(tf.shape(image)[0], tf.float64) * scale
    new_width = tf.cast(tf.shape(image)[1], tf.float64) * scale
    
    # Convert to float
    image = tf.image.convert_image_dtype(image, tf.float32)
    # Rescale by S
    image = tf.image.resize(image, size=(new_height, new_width), preserve_aspect_ratio=True)
    # Crop randomly
    image = tf.image.random_crop(image, size=[cut, cut, 3])
    # adjust RGB values by random amount
    image = tf.image.random_hue(image, max_delta=max_delta)
    
    # Split file path into parts
    parts = tf.strings.split(file_path, os.path.sep)
    # Extract class dir name
    one_hot = parts[-2] == class_names
    # Find index of maximum
    label = tf.argmax(one_hot)
    
    return image, label

In [3]:
import tensorflow as tf
from tensorflow import keras

# Generate SGD optimiser
def create_optimiser():
    
    sgd_optimiser = keras.optimizers.experimental.SGD(
            learning_rate=1e-2,
            momentum=0.9,
            nesterov=False,
            weight_decay=5e-4
        )
    
    return sgd_optimiser

In [4]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

# Build VGG11 model
def design_vgg11(class_num, input_shape, kernel_size, padding, activation, pool_size, strides):
    
    model = keras.models.Sequential([

        # 1st convolutional block
        layers.Conv2D(input_shape=input_shape, filters=64, kernel_size=kernel_size, padding=padding, activation=activation),
        layers.MaxPooling2D(pool_size=pool_size, strides=strides),

        # 2nd convolutional block
        layers.Conv2D(filters=128, kernel_size=kernel_size, padding=padding, activation=activation),
        layers.MaxPooling2D(pool_size=pool_size, strides=strides),

        # 3rd convolutional block
        layers.Conv2D(filters=256, kernel_size=kernel_size, padding=padding, activation=activation),
        layers.Conv2D(filters=256, kernel_size=kernel_size, padding=padding, activation=activation),
        layers.MaxPool2D(pool_size=pool_size, strides=strides),

        # 4th convolutional block
        layers.Conv2D(filters=512, kernel_size=kernel_size, padding=padding, activation=activation),
        layers.Conv2D(filters=512, kernel_size=kernel_size, padding=padding, activation=activation),
        layers.MaxPool2D(pool_size=pool_size, strides=strides),

        # 5th convolutional block
        layers.Conv2D(filters=512, kernel_size=kernel_size, padding=padding, activation=activation),
        layers.Conv2D(filters=512, kernel_size=kernel_size, padding=padding, activation=activation),
        layers.MaxPool2D(pool_size=pool_size, strides=strides),

        # Classifier head
        layers.Flatten(),
        layers.Dense(4096, activation=activation),
        layers.Dropout(rate=0.5),
        layers.Dense(4096, activation=activation),
        layers.Dropout(rate=0.5),
        layers.Dense(class_num)
    ])
    
    return model

In [5]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

# Build VGG13 model
def design_vgg13(model_file, kernel_size, padding, activation, pool_size, strides, layer_positions=[0, 2, 4, 5, 14, 16, 18]):
    
    # Import pretrained base
    pretrained_base = keras.models.load_model(model_file)
    
    # Select relevant layers
    pretrained_layers = [pretrained_base.get_layer(index=i) for i in layer_positions]
    
    model = keras.models.Sequential([

        # 1st convolutional block
        pretrained_layers[0],
        layers.Conv2D(filters=64, kernel_size=kernel_size, padding=padding, activation=activation),
        layers.MaxPooling2D(pool_size=pool_size, strides=strides),

        # 2nd convolutional block
        pretrained_layers[1],
        layers.Conv2D(filters=128, kernel_size=kernel_size, padding=padding, activation=activation),
        layers.MaxPooling2D(pool_size=pool_size, strides=strides),

        # 3rd convolutional block
        pretrained_layers[2],
        pretrained_layers[3],
        layers.MaxPool2D(pool_size=pool_size, strides=strides),

        # 4th convolutional block
        layers.Conv2D(filters=512, kernel_size=kernel_size, padding=padding, activation=activation),
        layers.Conv2D(filters=512, kernel_size=kernel_size, padding=padding, activation=activation),
        layers.MaxPool2D(pool_size=pool_size, strides=strides),

        # 5th convolutional block
        layers.Conv2D(filters=512, kernel_size=kernel_size, padding=padding, activation=activation),
        layers.Conv2D(filters=512, kernel_size=kernel_size, padding=padding, activation=activation),
        layers.MaxPool2D(pool_size=pool_size, strides=strides),

        # Classifier head
        layers.Flatten(),
        pretrained_layers[4],
        layers.Dropout(rate=0.5),
        pretrained_layers[5],
        layers.Dropout(rate=0.5),
        pretrained_layers[6],
    ])
    
    return model